In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch


In [2]:
cafef = {
    "Đời sống":
    ["https://cafef.vn/song.chn",
     "https://cafef.vn/lifestyle.chn", ],
    "Thời sự":
    ["https://cafef.vn/thoi-su.chn", ],
    "Chứng khoán":
    ["https://cafef.vn/thi-truong-chung-khoan.chn", ],
    "Bất động sản":
    ["https://cafef.vn/bat-dong-san.chn", ],
    "Doanh nghiệp":
    ["https://cafef.vn/doanh-nghiep.chn", ],
    "Tài chính":
    ["https://cafef.vn/tai-chinh-ngan-hang.chn",
     "https://cafef.vn/tai-chinh-quoc-te.chn",
     "https://cafef.vn/vi-mo-dau-tu.chn",
     "https://cafef.vn/kinh-te-so.chn",
     "https://cafef.vn/thi-truong.chn", ]
}


In [3]:
cafefLinks = {}
for key in cafef:
    cafefLinks[key] = []
for key, value in cafef.items():
    for url in value:
        try:
            response = requests.get(url)
            tree = lxml.html.fromstring(response.content)
            title_elem = tree.cssselect('h3 a')
            for elem in title_elem:
                cafefLinks[key].append("https://cafef.vn"+elem.get('href'))
        except:
            pass

for key in cafefLinks:
    print(key, len(cafefLinks[key]))

In [4]:
cafefContent = {}
for key in cafef:
    cafefContent[key] = []
for key, value in cafefLinks.items():
    for url in value:
        if(url.startswith('https://cafef.vn/')):
            try:
                id = url.split('-')[-1].replace('.chn', '')
                downloaded = trafilatura.fetch_url(url)
                tree = lxml.html.fromstring(str(downloaded))
                date_elem = tree.cssselect(".pdate")[0]
                content = trafilatura.extract(downloaded)
                content = content.replace('\n', ' ')
                item = {"domain": "https://cafef.vn", "URL": url,"content": content, "date": date_elem.text, "id": id}
                cafefContent[key].append(item)
            except:
                pass

for key in cafefContent:
    print(key, len(cafefContent[key]))


Đời sống 28
Thời sự 14
Chứng khoán 14
Bất động sản 14
Doanh nghiệp 14
Tài chính 70


In [5]:
client = Elasticsearch("http://localhost:9200")

client.info()


/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
def gendata():
    for key,value in cafefContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()